# Azure Batch 追加実行

## Nuget の実施

In [ ]:
#r "nuget: Microsoft.Azure.Batch"
#r "nuget: Azure.Storage.Blobs"
#r "nuget: Azure.Identity"
#r "nuget: DotNetEnv, 3.0.0"

### Batch 追加

### 宣言

In [ ]:
using Microsoft.Azure.Batch;
using Microsoft.Azure.Batch.Auth;
using Microsoft.Azure.Batch.Common;
using Microsoft.Extensions.Configuration;

DotNetEnv.Env.Load();

readonly string tenantId = DotNetEnv.Env.GetString("TENANT_ID");
readonly string location = DotNetEnv.Env.GetString("LOCATION");
readonly string BatchAccountName = DotNetEnv.Env.GetString("BATCH_ACCOUNT_NAME");
readonly string BatchAccountKey = DotNetEnv.Env.GetString("BATCH_ACCOUNT_KEY");
readonly string BatchAccountUrl = string.Format(DotNetEnv.Env.GetString("BATCH_ACCOUNTURL"), BatchAccountName, location);
// Storage account credentials
readonly string StorageAccountName = DotNetEnv.Env.GetString("STORAGE_ACCOUNT_NAME");
//const string StorageAccountKey = "";
// Batch resource settings
readonly string PoolId = DotNetEnv.Env.GetString("BATCH_POOL_ID");
readonly string JobId = DotNetEnv.Env.GetString("BATCH_JOB_ID");
readonly int PoolNodeCount = DotNetEnv.Env.GetInt("BATCH_POOL_NODE_COUNT");
readonly string PoolVMSize = DotNetEnv.Env.GetString("BATCH_POOL_VM_SIZE");

// Batch Image settings
readonly string publisher = DotNetEnv.Env.GetString("SHARED_IMAGE_PUBLISHER");
readonly string offer = DotNetEnv.Env.GetString("SHARED_IMAGE_OFFER");
readonly string sku = DotNetEnv.Env.GetString("SHARED_IMAGE_SKU");
readonly string version = DotNetEnv.Env.GetString("SHARED_IMAGE_VERSION");

### Batch JOB & タスクの追加

In [ ]:
try
{
    var cred = new BatchSharedKeyCredentials(BatchAccountUrl, BatchAccountName, BatchAccountKey);
    using BatchClient batchClient = BatchClient.Open(cred);
    var jobid = $"{JobId}add5";
    Console.WriteLine("Creating job [{0}]...", jobid);
    try
    {
        CloudJob job = batchClient.JobOperations.CreateJob();
        job.Id = jobid;
        job.PoolInformation = new PoolInformation { PoolId = PoolId };
        job.Commit();
    }
    catch (BatchException be)
    {
        Console.WriteLine(be);
        // Accept the specific error code JobExists as that is expected if the job already exists
        if (be.RequestInformation?.BatchError?.Code == BatchErrorCodeStrings.JobExists)
        {
            Console.WriteLine("The job {0} already existed when we tried to create it", jobid);
        }
        else
        {
            throw; // Any other exception is unexpected
        }
    }

    var tasks = new List<CloudTask>()
    {
        new CloudTask("Task1", "powershell -Command \"echo 'Hello World!'\"")
    };
    
    // Add all tasks to the job.
    batchClient.JobOperations.AddTask(jobid, tasks);
    // Monitor task success/failure, specifying a maximum amount of time to wait for the tasks to complete.
    TimeSpan timeout = TimeSpan.FromMinutes(30);
    Console.WriteLine("Monitoring all tasks for 'Completed' state, timeout in {0}...", timeout);
    IEnumerable<CloudTask> addedTasks = batchClient.JobOperations.ListTasks(jobid);
    batchClient.Utilities.CreateTaskStateMonitor().WaitAll(addedTasks, TaskState.Completed, timeout);
    Console.WriteLine("All tasks reached state Completed.");
    // Print task output
    Console.WriteLine();
    Console.WriteLine("Printing task output...");
    IEnumerable<CloudTask> completedtasks = batchClient.JobOperations.ListTasks(jobid);
    foreach (CloudTask task in completedtasks)
    {
        string nodeId = string.Format(task.ComputeNodeInformation.ComputeNodeId);
        Console.WriteLine("Task: {0}", task.Id);
        Console.WriteLine("Node: {0}", nodeId);
        Console.WriteLine("Standard out:");
        Console.WriteLine(task.GetNodeFile(Constants.StandardOutFileName).ReadAsString());
    }
}
catch(Exception e)
{
    Console.WriteLine(e);
}